# Building LLM-based applications for writting discharge summary

In this notebook, we will build an application to writting discharge summary.

For required packages, please run:

```bash
pip install -r requirements.txt
```

or

```bash
conda install --file requirements.txt
```

There are a few methods explored in this notebook:
1. Stuff with human and system prompt
2. Stuff with human prompt
3. Decompose method
4. MapReduce method
5. Refine method

In [1]:
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

load_dotenv()
llm = AzureChatOpenAI(
    deployment_name="gpt_35_16k",
    model_name="gpt-35-turbo-16k"
)

In [2]:
#!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
from IPython.display import Markdown

# Read cases

In [5]:
import glob
from prompting.read_data import read_markdown_file, read_json_file

file_list = glob.glob('./data/processed/merged/merged*.md')
# Filter those that are _original.md - ignore other languages for now
file_list = [f for f in file_list if '_original.md' in f]
work_with_file = file_list[0]

case = read_markdown_file(path=work_with_file)

# Print first 1000 characters of the case
Markdown(case[:1000] + '...')

# Medications, 2024-01-08
 * Norvasc (Amlodipine), PO, 5 Mg 1+0+0 times per day
 * Cozaar (Losartan), PO, 50 Mg 1+0+0 times per day
 * Fortamet (Metformin), PO, 500 Mg 2+0+2 times per day
 * Pfizerpen (Benzylpenicillin), IV, 1 G 0+0+1 times per day
 * Roxicodone (Oxicodone), PO, 5 Mg 0+0+1 times per day
 * Veetids (Phenoxymethylpenicillin), PO, 1 G 0+0+0 times per day

# Lab values, 2024-01-08, 18:26
 * B-Leukocytes 16.2 x10^9/L (3.5-8.8)
 * B-Erytrocytes 7.7 x10^12/L (3.9-5.2)
 * B-Hemoglobin 120 g/L (117-153)
 * B-EVF 0.45 U (0.35-0.46)
 * ERC(B)-MCV 92 fL (82-98)
 * ERC(B)-MCH 30 x10^9/L (27-33)
 * B-Trombocytes 178 x10^9/L (165-387)
 * P-Glucose 7.2 mmol/L (4.2-6.0)
 * P-Sodium 140 mmol/L (137-145)
 * P-Potassium 4.5 mmol/L (3.5-4.6)
 * P-CRP 279 mg/L (<3)
 * P-Creatinine 137 μmol/L (<90)

# ED admission note, 2024-01-08, 19:43, Dr A 

## Social security number
19550101–1111

## Reason for admittance
Swollen leg.

## Earlier/current diseases
Hypertension, diabetes mellitus type 2, ...

In [ ]:
from prompting.read_data import read_json_file

json_file_list = glob.glob('./data/processed/markdown*.json')
# Filter those that are _original.md - ignore other languages for now
json_file_list = [f for f in json_file_list if '_original.json' in f]

# Read and parse the json file as a dictionary
json_file = json_file_list[0]

case_object = read_json_file(json_file)
case_object

# Basic method (Naive)

In [ ]:
from prompting.basic import basic_chain

out = basic_chain(llm=llm).single.invoke({"notes": case})
Markdown(out)

In [ ]:
from prompting.basic import create_multiple_basic_prompts

stuff_df = create_multiple_basic_prompts(case=case, n=2, llm=llm)
stuff_df

In [ ]:
stuff_df.to_csv("data/output/stuff_df_final.csv")

# Decompose Technique

In [ ]:
from prompting.decompose import combine_all_sections

decompose_df = combine_all_sections(case=case_object, llm=llm, n = 2)

In [ ]:
decompose_df.to_csv("data/output/decompose_df_final.csv")

# Map Reduce Prompt Techniques

In [ ]:
from prompting.map_reduce import create_mapreduce_df

mapreduce_sum = create_mapreduce_df(case=case_object, n = 2)

In [ ]:
mapreduce_sum.to_csv("data/output/mapreduce_final.csv")

# Refine Prompt Techniques

In [ ]:
from prompting.refine import create_refine_df

refine_sum = create_refine_df(case=case_object, n=5, llm=llm)
refine_sum.to_csv("data/outputrefine_final.csv")